#  Using Matplotlib through C/C++ API


## 1 Python/C API

The Application Programmer’s Interface to Python gives C and C++ programmers access to the Python interpreter at a variety of levels. The API is equally usable from C++, but for brevity it is generally referred to as the Python/C API. There are two fundamentally different reasons for using the Python/C API. The first reason is to write extension modules for specific purposes; these are C modules that extend the Python interpreter. This is probably the most common use. The second reason is to use Python as a component in a larger application; this technique is generally referred to as embedding Python in an application.

https://docs.python.org/3/c-api/index.html

## 2 Example: matplotlib-cpp

https://github.com/lava/matplotlib-cpp

Extremely simple yet powerful header-only C++ plotting library built on the popular matplotlib 


### 2.1 Header and lib for using  matplotlib-cpp

**the header file only**

* matplotlib-cpp.h: `./demo/include/matplotlib-cpp`

**Python C API:**

* include

  * C:\Python37\include
  
  * C:\Python37\Lib\site-packages\numpy\core\include\

* Lib

 * C:\Python37


### 2.2  Complete minimal example

In [ ]:
%matplotlib inline 

import matplotlib.pyplot as plt

plt.figure(1) #create figure 1
plt.plot([1,2,3,4], [1,7,3,5]) #draw on figure 1 <x,y> list/array
plt.show() #show figure on screen

In [ ]:
%%file ./demo/src/demo_matplotlib.cpp
#include <iostream>
#include "../include/matplotlibcpp.h"

using namespace std;
namespace plt = matplotlibcpp;

int main() {
    vector<int> x{1,2,3,4};
    vector<double> y{1,7,3,5};
    plt::plot(x,y);
    plt::show();
}

In [ ]:
!g++ -std=c++11 -o ./demo/bin/demo.exe ./demo/src/demo_matplotlib.cpp  -IC:\Python37\include -IC:\Python37\Lib\site-packages\numpy\core\include\ -LC:\Python37\ -lpython37

* include: `-IC:\Python37\include -IC:\Python37\Lib\site-packages\numpy\core\include\`
        
* link lib: `-LC:\Python37\ -lpython37`

In [ ]:
%%file ./demo/makefile-demo_matplotlib.mk

INC= -IC:/Python37/include \
    -IC:/Python37/Lib/site-packages/numpy/core/include
        
LIBDIR= C:/Python37/

SRC= ./demo/src/demo_matplotlib.cpp 
            
BIN= ./demo/bin/demo

all: spring_matplotlib

spring_matplotlib: 
	 g++ -o $(BIN) $(SRC) $(INC) -L$(LIBDIR) -lPython37


In [ ]:
!make -f ./demo/makefile-demo_matplotlib.mk

![](./img/matplotlib-cpp.jpg)

### 2.3 The Behavior of Springs

[UNDERSTANDING_EXPERIMENTAL_DATA:UNDERSTANDING_EXPERIMENTAL_DATA](./Unit2-3-UNDERSTANDING_EXPERIMENTAL_DATA.ipynb)

In [6]:
%%file ./demo/data/springData.csv
Distance(m),Mass(kg)
0.0865,0.1
0.1015,0.15
0.1106,0.2
0.1279,0.25
0.1892,0.3
0.2695,0.35
0.2888,0.4
0.2425,0.45
0.3465,0.5
0.3225,0.55
0.3764,0.6
0.4263,0.65
0.4562,0.7
0.4502,0.75
0.4499,0.8
0.4534,0.85
0.4416,0.9
0.4304,0.95
0.437,1.0

Overwriting ./demo/data/springData.csv


In [4]:
%%file ./demo/src/spring_matplotlib.cpp

#include <iostream>
#include <fstream>
#include <sstream>
#include <string>
#include <vector>
#include "../include/matplotlibcpp.h"

using namespace std;

namespace plt = matplotlibcpp;

// Unit9-2-ctypes:Polynomial Regression
void PolynomialFit(double x[], double y[], int size, int n, double a[]);

int main(void)
{
  vector<double> vecforces;  
  vector<double> vecdistances; 
    
  ifstream fin("../data/springData.csv");
  if(!fin) {
     cerr<<"failed to open file for reading"<<endl;
     return 1;
  }
  string line;
  while(getline(fin, line))
  {
      istringstream sin(line);
      vector<string> fields; 
      string field;
      while (getline(sin, field, ','))
         fields.push_back(field); 
      string d =fields[0];
      string m =fields[1];
      if (atof(d.c_str()) > 0)
           vecdistances.push_back(atof(d.c_str()));
      if (atof(m.c_str()) > 0)
          vecforces.push_back(9.81*atof(m.c_str()));
  };
    
  fin.close();
 
  int n = 1; // n is the degree of Polynomial
  double a[n + 1];
  int size=vecdistances.size();
  double forces[size];
  double distances[size];
  copy(vecforces.begin(), vecforces.end(),forces);
  copy(vecdistances.begin(), vecdistances.end(),distances);
  PolynomialFit(forces, distances, size - 6, n, a);
  cout << "PolynomialFit:k =" << 1 / a[1] << std::endl;
  
  plt::plot(vecforces, vecdistances, "r*");
 
  vector<double> vecpredictedDistances;
  vector<double> vecprevarforces;
  for (int i = 0; i < size-6; i++)
  {  
     vecprevarforces.push_back(forces[i]);
     vecpredictedDistances.push_back(a[1]*forces[i] + a[0]);
  }    
  plt::plot(vecprevarforces, vecpredictedDistances,"b+");
  plt::plot(vecprevarforces, vecpredictedDistances);
    
  plt::title("Spring Data");
  plt::show();
}

Overwriting ./demo/src/spring_matplotlib.cpp


In [2]:
%%file ./demo/makefile-spring_matplotlib.mk

INC= -IC:/Python37/include \
    -IC:/Python37/Lib/site-packages/numpy/core/include
        
LIBDIR= C:/Python37/

SRC= ./demo/src/spring_matplotlib.cpp \
    ./demo/src/PolynomialFit.c 
            
BIN= ./demo/bin/demo

all: spring_matplotlib

spring_matplotlib: 
	 g++ -o $(BIN) $(SRC) $(INC) -L$(LIBDIR) -lPython37 

Overwriting ./demo/makefile-spring_matplotlib.mk


In [5]:
!make -f ./demo/makefile-spring_matplotlib.mk

g++ -o ./demo/bin/demo ./demo/src/spring_matplotlib.cpp ./demo/src/PolynomialFit.c  -IC:/Python37/include -IC:/Python37/Lib/site-packages/numpy/core/include -LC:/Python37/ -lPython37 


![](./img/spring_matplotlib_cpp.jpg)